# Forum 2

This notebook is designed for social scientists who want to explore how their own data might begin to interface with weather and hazard datasets.

### Goals:
1. Load and explore a sample of the Extreme Weather and Society Survey data
2. Bring in a sample weather dataset (e.g., watches/warnings/advisories from NWS)
3. Perform a simple merge
4. Visualize a relationship between survey responses and weather

Note: Ensure that the datasets are open access, or you have an API key for restricted access.  For this example, the datasets in this notebook do not require an API key.

### Why do this?

In general, the integration of weather and hazard data allows for a deeper understanding of how people perceive, interpret, and respond to weather and climate risks in relation to the actual meteorological conditions they experience.  A range of potential researchs can be posed:

1. How accurately do people perceive the frequency or severity of extreme weather events in their area?
2. Are there socioeconomic or demographic predictors of inaccurate weather risk perception?
3. Do people who recall receiving more weather warnings perceive greater weather risk?
4. Is there a mismatch between how meteorologists frame risk and how the public interprets it?
5. Are individuals who have experienced extreme weather more likely to change their behavior or beliefs?


### Import 

In Python, we import libraries to add extra tools that help us do things like make graphs, work with data, or do more complext calculations, like regression.  It's good practice to have all libraries imported at the beginning.  You can always add (even install) libraries as you go along, you just have to rerun the cells (or restart the kernel if a new install).

In [2]:
#import libraries
import requests
import pandas as pd
import time
from io import BytesIO
from datetime import timedelta
import geopandas as gpd
from shapely.geometry import Point
from io import StringIO
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import Counter
from shapely.geometry import Point
import seaborn as sns

## Step 1: Search for Datasets

By default, the Harvard Dataverse only returns 10 results per search request.  The first cell shows a search and retrieving only 10 results.  If you know there are more datasets available, the second cell shows to extend the search to retrieve all of the results.

##### Simple search for 10 results

In [11]:
# requires requests and pandas

# Define search query
query = "climate change"
search_url = f"https://dataverse.harvard.edu/api/search?q={query}&type=dataset"

# Perform search
response = requests.get(search_url)
results = response.json()

# Convert items to DataFrame
items = results['data']['items']
df_results = pd.DataFrame(items)

# Show key columns
#df_results[['name', 'global_id', 'published_at', 'citation']]
df_results.head(5)

,name,type,url,global_id,description,published_at,publisher,citationHtml,identifier_of_dataverse,name_of_dataverse,...,minorVersion,createdAt,updatedAt,contacts,authors,publications,image_url,keywords,producers,relatedMaterial
0,"Climate Change Helplessness, Pilot Study",dataset,https://doi.org/10.7910/DVN/HAEP2K,doi:10.7910/DVN/HAEP2K,Replication data for Pilot Study for Climate C...,2015-12-12T21:20:58Z,Climate Change Helplessness Dataverse,"Salomon, Erika, 2015, ""Climate Change Helpless...",cch,Climate Change Helplessness Dataverse,...,0,2015-12-12T21:20:24Z,2015-12-12T21:20:58Z,"[{'name': 'Salomon, Erika', 'affiliation': 'Un...","[Salomon, Erika]",NaN,NaN,NaN,NaN,NaN
1,"Climate Change Helplessness, Study 1",dataset,https://doi.org/10.7910/DVN/OVZLG5,doi:10.7910/DVN/OVZLG5,Replication data for Study 1 of Climate Change...,2015-10-10T22:12:30Z,Climate Change Helplessness Dataverse,"Salomon, Erika, 2015, ""Climate Change Helpless...",cch,Climate Change Helplessness Dataverse,...,0,2015-06-26T20:32:33Z,2015-10-10T22:12:30Z,"[{'name': 'Salomon, Erika', 'affiliation': 'Un...","[Salomon, Erika]",NaN,NaN,NaN,NaN,NaN
2,"Climate Change Helplessness, Study 2",dataset,https://doi.org/10.7910/DVN/8Z7M4G,doi:10.7910/DVN/8Z7M4G,Replication data for Study 2 of Climate Change...,2015-10-10T22:13:10Z,Climate Change Helplessness Dataverse,"Salomon, Erika, 2015, ""Climate Change Helpless...",cch,Climate Change Helplessness Dataverse,...,0,2015-06-26T20:34:45Z,2015-10-10T22:13:10Z,"[{'name': 'Salomon, Erika', 'affiliation': 'Un...","[Salomon, Erika]",NaN,NaN,NaN,NaN,NaN
3,"Climate Change Helplessness, Study 3",dataset,https://doi.org/10.7910/DVN/KXW0LE,doi:10.7910/DVN/KXW0LE,Replication data for Study 2 of Climate Change...,2015-10-10T22:13:58Z,Climate Change Helplessness Dataverse,"Salomon, Erika, 2015, ""Climate Change Helpless...",cch,Climate Change Helplessness Dataverse,...,0,2015-06-26T20:42:33Z,2015-10-10T22:13:58Z,"[{'name': 'Salomon, Erika', 'affiliation': 'Un...","[Salomon, Erika]",NaN,NaN,NaN,NaN,NaN
4,3_QGIS file with island and climate change ris...,dataset,https://doi.org/10.7910/DVN/O6G3AI,doi:10.7910/DVN/O6G3AI,QGIS file to visualise and analyse climate cha...,2022-10-28T10:09:02Z,I Climate Change Risk Assessment,"Lammers, Katrin; Gerbatsch, Karoline, 2022, ""3...",ClimRisk,I Climate Change Risk Assessment,...,0,2022-10-24T14:38:34Z,2022-10-28T10:09:02Z,"[{'name': 'Lammers, Katrin', 'affiliation': 'R...","[Lammers, Katrin, Gerbatsch, Karoline]",[{}],NaN,NaN,NaN,NaN


##### Search with more than 10 results

In [8]:
query = "ripberger"
start = 0
per_page = 100  # Max per page is 100
all_items = []

while True:
    search_url = (
        f"https://dataverse.harvard.edu/api/search?"
        f"q={query}&type=dataset&start={start}&per_page={per_page}"
    )
    
    response = requests.get(search_url)
    data = response.json()
    
    items = data['data']['items']
    all_items.extend(items)
    
    # Break if fewer than per_page results are returned (i.e., last page)
    if len(items) < per_page:
        break
    start += per_page

# Convert to DataFrame
df_results = pd.DataFrame(all_items)
df_results[['name', 'global_id', 'published_at', 'citation']].head(5)

,name,global_id,published_at,citation
0,A comprehensive inventory and review of color ...,doi:10.7910/DVN/IFBAZ4,2025-04-22T13:26:32Z,"Ripberger, Joseph; Bitterman, Abby; Rosen, Zoe..."
1,WX18,doi:10.7910/DVN/RHT4ON,2020-02-03T22:11:23Z,"Ripberger, Joseph; Silva, Carol; Jenkins-Smith..."
2,WX19,doi:10.7910/DVN/MLCJEW,2020-02-03T22:10:59Z,"Ripberger, Joseph; Silva, Carol; Jenkins-Smith..."
3,WX17,doi:10.7910/DVN/GSTYK4,2020-02-03T22:10:34Z,"Ripberger, Joseph; Silva, Carol; Jenkins-Smith..."
4,WX20,doi:10.7910/DVN/EWOCUA,2020-09-01T14:03:39Z,"Ripberger, Joseph; Krocak, Makenzie; Silva, Ca..."


In [12]:
#How many datasets?
df_results.shape

(10, 28)

### Subsetting Your Results

Let's say you don't want all of these, but only a subset of related surveys.  For this exercise and the remainder of notebook, we will focus on the yearly waves of the Extreme Weather and Society Survey (WXYY). So let's subset them.

In [ ]:
# Define the dataset names you want to filter on
target_names = ["WX17", "WX18", "WX19", "WX20", "WX21", "WX22", "WX23", "WX24"]

# Subset the dataframe
df_subset = df_results[df_results['name'].isin(target_names)]

# Display the result
df_subset

## Step 2: Get Dataset Metadata and Files

### Getting metadata for a single dataset

Let's examine the metadata for one of the datasets in the subset, WX18.

In [ ]:
# Extract persistent ID (DOI) from the first row [0] by position
persistent_id = df_subset.iloc[0]['global_id']

# Get dataset metadata
metadata_url = f"https://dataverse.harvard.edu/api/datasets/:persistentId/?persistentId={persistent_id}"
metadata_response = requests.get(metadata_url).json()

# Display list of files
files = metadata_response['data']['latestVersion']['files']
files


### Getting metadata for multiple datasets

Let's say we want the metadata for all waves.  This cell is set up as a "loop", that is, it will loop through all of the files and pick up the metadata information we want.  This time, we will limit the items we want to see (as seen in the "for f in files" loop below) and have the results formatted into a dataframe.

In [ ]:
# Create an empty list to hold all file metadata
all_files = []

# Loop through persistent IDs
for pid in df_subset['global_id']:
    metadata_url = f"https://dataverse.harvard.edu/api/datasets/:persistentId/?persistentId={pid}"
    response = requests.get(metadata_url)
    
    if response.status_code == 200:
        metadata = response.json()
        files = metadata['data']['latestVersion']['files']
        
        for f in files:
            file_info = {
                'dataset_title': metadata['data']['latestVersion']['metadataBlocks']['citation']['fields'][0]['value'],
                'file_id': f['dataFile']['id'],
                'file_label': f['label'],
                'file_size': f['dataFile'].get('filesize', None),
                'file_description': f.get('description', ''),
                'persistent_id': pid
            }
            all_files.append(file_info)
    
    # Be respectful of API limits
    time.sleep(0.5)

# Convert to DataFrame
df_files = pd.DataFrame(all_files)
df_files

## Step 3: Download a File

Above, we see that each dataset file (.tab) is accompanied by PDFs of the instrument and a reference report.

Let's download the first year of the survey, WX18, which has a file_id of "3657710".

In [ ]:
# File ID for WxEM_Wave1.tab
file_id = 3657710

# Download directly to memory
file_url = f"https://dataverse.harvard.edu/api/access/datafile/{file_id}?format=original"
response = requests.get(file_url)

# Load into pandas directly from memory, assuming comma-delimited content
df_18 = pd.read_csv(BytesIO(response.content), sep=',', encoding='ISO-8859-1', engine='python', on_bad_lines='skip')
df_18.head()

### Examine the dataset

Pandas has a number of attributes that can be used to examine characteristics of the dataset.

In [ ]:
# See basic shape of the data (rows, columns)
df_18.shape

In [ ]:
# List all column names
df_18.columns.tolist()

#### Variable names

Listing the columns shows all the variable names contained in the data.  To integrate with weather data, we are most interested in locating possible ways to join the data.  Typically, geographic variables are a good start.  In this survey, some good possible variables are state, zip, lat/lon.  These are pretty straightforward, but what about "nws_region"?  Does it contain WFOs?  Let's examine.

In [ ]:
#Is nws_region usefl at all?
df_18['nws_region'].unique()

##### Unfortunately, "nws_region" only has four regions.  Nonetheless, it might be useful at some point.

## Step 4: Merge with Weather Data

In this example, we will extract warnings/watches/advisories from the Iowa Mesonet for each respondent three days prior to the start of the survey.  I know, that's pretty arbitrary but perhaps we're interested in the impact of the weather on responses to a weather survey.

For each respondent, we will use their:

1. lat, lon, and begin_date
2. Query the IEM for any warnings/watches/advisories issued within 3 days prior to begin_date
3. Store or summarize those results (e.g., number of WWAs, types, text, etc.)

Please note the API Endpoint (Rest-like) and the structure of the request for those variables:

https://mesonet.agron.iastate.edu/vtec/json.php?lon={lon}&lat={lat}&sdate={start}&edate={end}

We will use this API call to loop through all 3,000 respondents and gather their information.

### Iowa Mesonet 

So I'm just going to use lat/lon and the date to directly get watches/warnings/advisories.  

In [ ]:
# Make sure begin_date is in datetime format
df_18['begin_date'] = pd.to_datetime(df_18['begin_date'])

# New column to store list of WWA names
df_18['wwa_names'] = None

# Loop through each respondent
for idx, row in tqdm(df_18.iterrows(), total=len(df_18)):
    lat = row['lat']
    lon = row['lon']
    end_date = row['begin_date']
    start_date = end_date - timedelta(days=3)

    # Build API URL
    url = (
        f"https://mesonet.agron.iastate.edu/json/vtec_events_bypoint.py"
        f"?lat={lat}&lon={lon}&sdate={start_date.strftime('%Y-%m-%d')}&edate={end_date.strftime('%Y-%m-%d')}"
    )

    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            names = [event['name'] for event in data.get('events', [])]
            df_18.at[idx, 'wwa_names'] = names
        else:
            df_18.at[idx, 'wwa_names'] = []
    except Exception as e:
        print(f"Failed for idx={idx}, lat={lat}, lon={lon}: {e}")
        df_18.at[idx, 'wwa_names'] = []

In [ ]:
#let's examine what we just collected.  We use the attribute "dropna" to make sure that there are no rows that empty (i.e., we didn't screw something up).
df_18[['lat', 'lon', 'begin_date', 'wwa_names']].dropna().head(10)

In [ ]:
#let's make sure we have the same number of row that we started with
df_18.shape

In [ ]:
#how many respondents experienced watches/warnings/advisories?
df_18['wwa_names'].apply(lambda x: isinstance(x, list) and len(x) > 0).sum()

In [ ]:
#who received a tornado warning?
df_18[df_18['wwa_names'].apply(lambda x: 'Tornado Warning' in x if isinstance(x, list) else False)]

In [ ]:
#who received more than two?
df_18[
    df_18['wwa_names'].apply(lambda x: isinstance(x, list) and len(x) > 2)
][['lat', 'lon', 'begin_date', 'wwa_names']]

### Quick Visualizations

These are quick descriptive visualizations.

In [ ]:
# Step 1: Ensure lat/lon are float (just in case)
df_18['lat'] = pd.to_numeric(df_18['lat'], errors='coerce')
df_18['lon'] = pd.to_numeric(df_18['lon'], errors='coerce')

# Step 2: Create geometry from lat/lon
geometry = [Point(xy) for xy in zip(df_18['lon'], df_18['lat'])]
gdf_18 = gpd.GeoDataFrame(df_18, geometry=geometry, crs="EPSG:4326")

# Step 3: Count WWAs
gdf_18['wwa_count'] = gdf_18['wwa_names'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Step 4: Plot
fig, ax = plt.subplots(figsize=(10, 6))
gdf_18.plot(column='wwa_count', cmap='OrRd', legend=True, ax=ax, markersize=10)
ax.set_title("Survey Respondents by Number of WWAs (3 Days Prior)", fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Flatten and count
all_names = df_18['wwa_names'].dropna().explode()
top_names = Counter(all_names).most_common(10)

# Plot
names, counts = zip(*top_names)
plt.figure(figsize=(10, 5))
plt.barh(names[::-1], counts[::-1])
plt.title("Top 10 Most Frequent WWAs")
plt.xlabel("Number of Respondents Exposed")
plt.tight_layout()
plt.show()

In [ ]:
# Count WWAs per date
timeline = (
    df_18[['begin_date', 'wwa_names']]
    .dropna()
    .assign(wwa_count=lambda df: df['wwa_names'].apply(len))
    .groupby('begin_date')['wwa_count']
    .sum()
)

# Plot
timeline.plot(marker='o', figsize=(10, 4), title='Total WWAs by Survey Date')
plt.ylabel('Total Warnings/Watch Events')
plt.xlabel('Survey Date')
plt.grid(True)
plt.tight_layout()
plt.show()

### Visualizing WWWAs by Survey Reponses

For a next step, we might do some quick analyses/visualizations of WWAs by survey response.  Perhaps even make an interactive version with a dropdown of survey responses that a user can play with?

Let's say we want to examine how the presence of recent NWS advisories (3 days prior to taking the survey) correlates with survey respondents' perceived risk of that hazard.  How can we do that?

Step 1.  Create two groups of respondents, those experienced a WWA prior to the survey and those who did not.
Step 2.  Extract risk perception scores for a hazard (For risk_flood: 1-No risk, 2-Low Risk.....5-Extreme risk)
Step 3.  Create a comparative visualization




In [ ]:
# Define relevant flood WWA keywords
flood_wwa_keywords = ['Flood Advisory', 'Flood Warning', 'Flash Flood Warning', 'Flash Flood Watch', 'Flood Watch']

# Create binary exposure column
df_18['flood_wwa_exposure'] = df_18['wwa_names'].apply(
    lambda wwas: any(flood_type in wwas for flood_type in flood_wwa_keywords)
)

In [ ]:
# Define correct order for risk perception labels
ordered_risk_labels = ['No risk', 'Low risk', 'Moderate risk', 'High risk', 'Extreme risk']

# Create the plot
plt.figure(figsize=(10, 6))
ax = sns.countplot(
    data=df_18,
    x='risk_flood_label',
    hue='flood_wwa_exposure',
    order=ordered_risk_labels
)

# Calculate total count per category to get percentages
total_counts = df_18.groupby(['risk_flood_label', 'flood_wwa_exposure']).size().reset_index(name='count')
category_totals = df_18.groupby('risk_flood_label').size()

# Add percentage labels on each bar
for p in ax.patches:
    height = p.get_height()
    if height == 0:
        continue  # skip zero-height bars
    x = p.get_x() + p.get_width() / 2
    hue_val = p.get_facecolor()  # used just for clarity, not necessary
    label = p.get_label()
    category = p.get_x() + p.get_width() / 2
    risk_label = p.get_x()
    
    # Get the category label (x-axis tick) associated with this bar
    idx = int(p.get_x() + p.get_width() / 2)
    category = ordered_risk_labels[int(p.get_x() + p.get_width() / 2)]
    
    # Get percent value
    bar_label = ax.get_xticklabels()[int(x)].get_text()
    total = category_totals[bar_label]
    percent = 100 * height / total
    ax.text(x, height + 1, f'{percent:.1f}%', ha='center', va='bottom', fontsize=9)

# Final plot touches
plt.xlabel("Perceived Flood Risk")
plt.ylabel("Number of Respondents")
plt.title("Perceived Flood Risk by Exposure to Flood-Related WWAs")
plt.legend(title="WWA Exposure", labels=["No", "Yes"])
plt.tight_layout()

# Save to PNG.  It will be saved in the directory with this notebook.
plt.savefig("perceived_flood_risk_by_wwa.png", dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
# Calculate percentages
crosstab = pd.crosstab(df_18['flood_wwa_exposure'], df_18['risk_flood_label'], normalize='index')

# Plot
crosstab.loc[[True, False]].reindex(columns=ordered_risk_labels).T.plot(
    kind='bar', stacked=True, figsize=(10, 6), colormap='viridis'
)
plt.title('Flood Risk Perception by WWA Exposure (Proportions)')
plt.xlabel('Perceived Flood Risk')
plt.ylabel('Proportion of Respondents')
plt.legend(title='Exposed to Flood WWA', labels=['Yes', 'No'])
plt.tight_layout()

# Save to PNG.  It will be saved in the directory with this notebook.
plt.savefig("perceived_flood_risk_by_wwa.png", dpi=300, bbox_inches='tight')

plt.show()


In [ ]:
# I don't personally like violin plots, but let's take a look
sns.violinplot(data=df_18, x='flood_wwa_exposure', y='risk_flood', inner='quartile')
plt.xticks([0, 1], ['No WWA Exposure', 'WWA Exposure'])
plt.xlabel("Exposure to Flood WWA")
plt.ylabel("Perceived Flood Risk (1-5)")
plt.title("Distribution of Flood Risk Perception by WWA Exposure")
plt.tight_layout()
plt.show()


## Going further

We're not limited to visualizations, we could explore regressions, structural equation modeling, etc.  But perhaps it is enough that we've shown how to enrich a dataset that could be saved and used in whatever statistics package they like: R, SAS, SPSS, Stata, etc.